## Initialization

In [139]:
from copy import deepcopy
import numpy as np
from collections import defaultdict
from IPython.display import clear_output
import time
import random
import ttt as env

STATES = 19683
ACTIONS = 9

Q1 = defaultdict(lambda:np.array([0.6]*ACTIONS))
Q2 = deepcopy(Q1)

EPISODES = 5000 # how many times to run the enviornment from the beginning
MAX_STEPS = 20  # max number of steps allowed for each run of enviornment

LEARNING_RATE = 0.81  # learning rate
GAMMA = 0.96

RENDER = False # if you want to see training set to true

epsilon = 0.9

In [137]:
from importlib import reload
reload(env)

<module 'ttt' from 'd:\\Projects\\TicTacToe_AI\\ttt.py'>

## Training

In [140]:
rewards1 = []
rewards2 = []
for episode in range(EPISODES):
  if (episode+1) % 1000 == 0:
    clear_output()
    print(f"{(episode+1)} / {EPISODES} {((episode+1)/EPISODES)*100:.2f}%", flush=True)

  state = env.reset_game()
  for _ in range(MAX_STEPS):

    if np.random.uniform(0, 1) < epsilon:
      action = env.get_action()
    else:
      action = np.argmax(Q1[state])

    next_state, winner, reward = env.take_action(action)

    Q1[state][action] = Q1[state][action] + LEARNING_RATE * \
        (reward + GAMMA * np.max(Q1[next_state]) - Q1[state][action])

    x_state, x_action = state, action

    if winner and winner != "Draw":
      rewards1.append(reward)
      Q2[o_state][o_action] = Q2[o_state][o_action] + LEARNING_RATE * \
          (-1.0 + GAMMA * np.max(Q2[state]) - Q2[o_state][o_action])
      rewards2.append(-1.0)
      epsilon -= 0.001
      break  # reached goal
    elif winner == "Draw":
      rewards1.append(reward)
      Q2[o_state][o_action] = Q2[o_state][o_action] + LEARNING_RATE * \
          (0.5 + GAMMA * np.max(Q2[state]) - Q2[o_state][o_action])
      rewards2.append(0.5)
      epsilon -= 0.001
      break  # reached goal

    state = next_state

    if np.random.uniform(0, 1) < epsilon:
      action = env.get_action()
    else:
      action = np.argmax(Q2[state])

    next_state, winner, reward = env.take_action(action)

    Q2[state][action] = Q2[state][action] + LEARNING_RATE * \
        (reward + GAMMA * np.max(Q2[next_state]) - Q2[state][action])

    o_state, o_action = state, action

    if winner:
      rewards2.append(reward)
      Q1[x_state][x_action] = Q1[x_state][x_action] + LEARNING_RATE * \
          (-1.0 + GAMMA * np.max(Q1[state]) - Q1[x_state][x_action])
      rewards1.append(-1.0)
      epsilon -= 0.001
      break  # reached goal

    state = next_state

# print(Q1)
print(f"Average reward for X: {sum(rewards1)/len(rewards1)}: {rewards1[-10:-5]}")
print(f"Average reward for O: {sum(rewards2)/len(rewards2)}: {rewards2[-10:-5]}")
# and now we can see our Q values!


5000 / 5000 100.00%
Average reward for X: 0.8854770954190838: [1.0, 1.0, 1.0, 1.0, 1.0]
Average reward for O: -0.8789757951590318: [-1.0, -1.0, -1.0, -1.0, -1.0]


In [133]:
# Save Q
fileName = "q1.npy"
pathToFile = "Q_Tables/"+fileName
data = dict(Q1)
np.save(pathToFile, data)
fileName = "q2.npy"
pathToFile = "Q_Tables/"+fileName
data = dict(Q2)
np.save(pathToFile, data)